In [1]:
from functions import *
import logging
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

logging.getLogger().addHandler(logging.NullHandler())
logging.getLogger("natten.functional").setLevel(logging.ERROR)
device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")

print("Device:", device)

DATASET_PATH = "../data"
CHECKPOINT_PATH = "../VIT/EMPR"

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")
print("Device:", device)

wt_dc = 0.06

model_type = 'Dinat_Superloss'

fourclass = True
Speaker_Disentanglement = False
Entropy = False
pretrain = False
pretraining = False
new_test = False 

if pretrain==True:
    pathstr = r'/media/carol/Data/Documents/Emo_rec/NewMel/Pretraining/Arousal'
    # pathstr = r"/media/carol/Data/Documents/Emo_rec/Trained Models/DINAT/IEMOCAP/Pre_SD_wtd_006_l"
    
    processor_path = os.path.join(pathstr, 'processor')
    model_path = os.path.join(pathstr, 'model')
    # model_path = os.path.join('G:\My Drive\MaSc\emo_rec\MODELS\For Paper\IEMOCAP\LABEL_CURRIUCULUM\PreSD_012', "model")
elif model_type != 'ViT':
    model_path = 'shi-labs/dinat-mini-in1k-224'
    processor_path = 'shi-labs/dinat-mini-in1k-224'
    pathstr = model_path
else:
    model_path = 'google/vit-base-patch16-224-in21k'
    processor_path = 'google/vit-base-patch16-224-in21k'
    pathstr = model_path

/media/carol/Data/Documents/Emo_rec/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Device: cuda:0
Device: cuda:0


<Figure size 640x480 with 0 Axes>

In [2]:
EMOTIONS = {
    0: 'neutral',
    1: 'happy',
    2: 'sad',
    3: 'angry',
}
Map2Num = {
    'neutral': 0,
    'happy': 1,
    'sad': 2,
    'angry': 3,
}
Cval = 4


train_size = 20
eval_size = 20

In [3]:
from transformers import TrainingArguments, Trainer,  SchedulerType

metric_name = "eval_uar"
args = TrainingArguments(
    f"./logs2",
    save_strategy="epoch",
    evaluation_strategy="epoch",
    disable_tqdm=True,  # This disables the default progress bar
    learning_rate=1e-5,
    lr_scheduler_type=SchedulerType.COSINE_WITH_RESTARTS,
    warmup_ratio=0.1,
    per_device_train_batch_size=train_size,
    per_device_eval_batch_size=eval_size,
    num_train_epochs=50,
    weight_decay=wt_dc,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
    logging_dir='./logs_DO_NOT_DELETE/3090',
    remove_unused_columns=False,
    # logging_strategy="epoch",  # Log at the end of each epoch

)

    

In [4]:
dataset_train = 'cairocode/IEMO_Mel_6' #'cairocode/IEMO_007_NOSPLIT'
train_d0 = load_dataset(dataset_train, split='train')

dataset_val = 'cairocode/MSPI_Mel6'#'cairocode/MSPI_007_NOSPLIT_an'
val_dataset0 = load_dataset(dataset_val, split='train')
val_dataset0  = val_dataset0.filter(filter_m_examples)

In [5]:
ds_tr = os.path.split(dataset_train)[1]
ds_vl = os.path.split(dataset_val)[1]


In [6]:


root_path = os.path.join(r'/media/carol/Data/Documents/Emo_rec/NewMel', ds_tr)
root_path = create_unique_output_dir(root_path)
print(root_path)

/media/carol/Data/Documents/Emo_rec/NewMel/IEMO_Mel_6/20250128_2


In [ ]:
 
from sklearn.metrics import accuracy_score, balanced_accuracy_score
from collections import defaultdict
# Example usage

log_file = os.path.join(root_path,"training_logs.csv")
fieldnames = ["timestamp", "log"]
sys.stdout = CSVLogger(log_file, fieldnames)

xcorp_results = {
    'y_true': [],
    'y_pred': {f'run_{i}': [] for i in range(10)}
}
spkrs = [sample['speakerID'] for sample in train_d0]
unique_speakers = list(set(spkrs))
print(unique_speakers)

all_y_true = defaultdict(list)
all_y_pred = defaultdict(list)

angry_weight = 1
happy_weight = 5
neutral_weight = 2
sad_weight = 1

cw_dict = {}
cw_dict[0] = [1.1,1.5,1,1] #CONFIRMED
cw_dict[1] = [1.1,1.5,1,1] #CONFIRMED

cw_dict[2] = [0.97,1.5,1,1]
cw_dict[3] = [1,1.6,1,1]  #CONFIRMED

cw_dict[4] = [0.97,1.7,1,1]

cw_dict[5] = [0.95,1.7,1,1]

cw_dict[6] = [1.05,1.7,1,1]

cw_dict[7] = [1,1.8,0.93,1] #CONFIRMED
cw_dict[8] = [1,1.5,0.92,1] 

cw_dict[9] = [0.95,2.5,0.95,1] 


# class_weights = [2, 6.5,1,1]

class_weight_multipliers = {
    0: neutral_weight,
    1: happy_weight,
    2: sad_weight,
    3: angry_weight
}
total_results = pd.DataFrame()

for i in range (len(unique_speakers)):
    
    # speakers = [unique_speakers[i]]  #    speakers = [937+i]
    speakers = [11-unique_speakers[i]]

    num = speakers[0]-1
    print(f"NUM = {num} __ SPKRS {speakers}")
    print(f"\n {'#'*120}")
    print(f"                                          STARTING SPEAKER {num}                                                      ")
    print(f"\n {'#'*120}")

    new_model_path = os.path.join(root_path, str(num))
    os.makedirs(new_model_path, exist_ok=True)
    
    # Create the test split
    test_dataset = train_d0.filter(lambda x: x['speakerID'] in speakers).filter(filter_m_examples)
    
    # Create the remaining data
    train_set = train_d0.filter(lambda x: x['speakerID'] not in speakers).filter(filter_m_examples)

    # train_set = train_set.train_test_split(test_size = 0.2)
    train_dataset = train_set #['train']
    val_dataset = test_dataset #train_set['test']
    # val_dataset = concatenate_datasets([val_dataset0, test_dataset])

    processor = AutoImageProcessor.from_pretrained(processor_path)

    base_model = DinatForImageClassification.from_pretrained(
        model_path,
        id2label=EMOTIONS,
        num_labels=Cval,
        label2id=Map2Num,
        ignore_mismatched_sizes=True,
        problem_type='single_label_classification'
    )
    if Speaker_Disentanglement == True:
        custom_dataset = CustomDataset(train_dataset)
        custom_sampler = CustomSampler(custom_dataset)
        train_dataloader = DataLoader(train_dataset, sampler=custom_sampler, collate_fn=collate_fn, batch_size=train_size)
    else:
        train_dataloader = DataLoader(train_dataset, collate_fn=collate_fn, batch_size=train_size)

    # class_weights = calculate_class_weights(train_dataset, class_weight_multipliers)
    # class_weights = [1.1,1.8,0.93,1]
    # class_weights = [neutral]
    class_weights = cw_dict[num]
    class_weights = torch.tensor(class_weights, dtype=torch.float).to(device)
    model = CustomDinatForImageClassification(base_model, num_classes=Cval, class_weights= class_weights)#, initial_class_weights=class_weights)

    model.to(device)
    super_loss_params = {
    'C': Cval,  # Example value, adjust based on your needs
    'lam': 0.01,  # Example value, adjust based on your needs
    'batch_size': args.train_batch_size,  # Pass the batch size dynamically
    'class_weights':class_weights
    }

    
    val_dataset.set_transform(val_transforms)
    val_dataset0.set_transform(val_transforms)
    train_dataset.set_transform(train_transforms)

    # early_stopping = EarlyStoppingCallback(early_stopping_patience=10, early_stopping_threshold=0.001)
    early_stopping = EarlyStoppingCallback(early_stopping_patience=12, early_stopping_threshold=0.001)

    trainer = CustomTrainer(
        model=model,
        args=args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        data_collator=collate_fn,
        compute_metrics=compute_metrics,
        tokenizer=processor,
        callbacks=[early_stopping]#, ClassWeightLoggerCallback()],  # Add the new callback here
        # super_loss_params=super_loss_params,  # Pass the custom loss parameters here
        )
    trainer.train()
    trainer.save_model(os.path.join(new_model_path, "model"))
    processor.save_pretrained(os.path.join(new_model_path, "processor"))
    test_dataset.set_transform(test_transforms)
    outputs = trainer.predict(test_dataset)
    print(outputs.metrics['test_accuracy']*100, "\t", outputs.metrics['test_uar']*100)
    print(outputs.metrics)
    model_info = {
        "Pretrain_file": pathstr,
        "Dataset Used": dataset_train,
        "Model Type": model_type,
        "Super Loss PARAMS": super_loss_params,
        "Speaker Disentanglement": Speaker_Disentanglement,
        "Entropy Curriculum Training": Entropy,
        "Column Trained on": column,
        "Test Results": outputs.metrics,
        "Test SpeakerID": speakers,
        "Class Weight": cw_dict[num],
        "NUM":  num,
        "Weight decay": wt_dc
    }
    file_path = save_model_header(new_model_path, model_info)
    matrix_path = save_confusion_matrix(outputs, dataset_train, new_model_path, Map2Num)
    dataset_test = dataset_val
    outputs2 = trainer.predict(val_dataset0)
    # print(outputs2.metrics)
    # print(outputs2.metrics['test_accuracy']*100, "\t", outputs2.metrics['test_uar']*100)

    new_row = {f'{ds_tr}_ACC': outputs.metrics['test_accuracy']*100, f'{ds_tr}_UAR': outputs.metrics['test_uar']*100, f'{ds_vl}_ACC': outputs2.metrics['test_accuracy']*100,f'{ds_vl}_UAR': outputs2.metrics['test_uar']*100}
    print(f"\n {'-'*120}")
    print("\n\n", new_row, "\n")
    
    # Method 1: Using concat()
    total_results = pd.concat([total_results, pd.DataFrame([new_row])], ignore_index=True)

    if i == 0:
        xcorp_results['y_true'].extend(outputs2.label_ids)

    # Store y_pred for each run
    xcorp_results['y_pred'][f'run_{i}'].extend(outputs2.predictions.argmax(axis=1))


    matrix_path = save_confusion_matrix(outputs2, dataset_test, new_model_path, Map2Num)
    all_y_true[ds_tr].extend(outputs.label_ids)
    all_y_pred[ds_tr].extend(outputs.predictions.argmax(axis=1))
    all_y_true[ds_vl].extend(outputs2.label_ids)
    all_y_pred[ds_vl].extend(outputs2.predictions.argmax(axis=1))



    torch.cuda.empty_cache()
    del trainer
    del model

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
NUM = 9 __ SPKRS [10]

 ########################################################################################################################
                                          STARTING SPEAKER 9                                                      

 ########################################################################################################################


/media/carol/Data/Documents/Emo_rec/.venv/lib/python3.10/site-packages/huggingface_hub/file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration. Please open a PR/issue to update `preprocessor_config.json` to use `image_processor_type` instead of `feature_extractor_type`. This warning will be removed in v4.40.
Some weights of DinatForImageClassification were not initialized from the model checkpoint at shi-labs/dinat-mini-in1k-224 and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([1000, 512]) in the checkpoint and torch.Size([4, 512]) in the model instantiated
- classifier.bias: found shape torch.Si

{'eval_loss': 1.3160279989242554, 'eval_accuracy': 0.4009216589861751, 'eval_uar': 0.28902101166371635, 'eval_f1': 0.21610827524520387, 'eval_top_k_acc': 0.5944700460829493, 'eval_runtime': 1.6621, 'eval_samples_per_second': 261.119, 'eval_steps_per_second': 13.236, 'epoch': 1.0}
{'eval_loss': 1.1933103799819946, 'eval_accuracy': 0.49539170506912444, 'eval_uar': 0.39946404586945894, 'eval_f1': 0.363358123569794, 'eval_top_k_acc': 0.7534562211981567, 'eval_runtime': 1.6496, 'eval_samples_per_second': 263.095, 'eval_steps_per_second': 13.337, 'epoch': 2.0}
{'loss': 1.2607, 'grad_norm': 37.41012954711914, 'learning_rate': 4.926108374384237e-06, 'epoch': 2.4630541871921183}
{'eval_loss': 1.1048977375030518, 'eval_accuracy': 0.5506912442396313, 'eval_uar': 0.4706426049002358, 'eval_f1': 0.4654629535733865, 'eval_top_k_acc': 0.7903225806451613, 'eval_runtime': 1.6902, 'eval_samples_per_second': 256.771, 'eval_steps_per_second': 13.016, 'epoch': 3.0}
{'eval_loss': 1.0351556539535522, 'eval_ac

KeyboardInterrupt: 

In [8]:
class_weights

tensor([1.1000, 1.5000, 1.0000, 1.0000], device='cuda:0')

In [ ]:
class_weights

tensor([1.1000, 1.5000, 1.0000, 1.0000], device='cuda:0')

In [ ]:
class_weights

tensor([1.1000, 1.5000, 1.0000, 1.0000], device='cuda:0')

In [11]:
# If you want to get the final prediction based on majority voting across all runs
y_true = np.array(xcorp_results['y_true'])

final_prediction = np.array([np.bincount([xcorp_results['y_pred'][f'run_{i}'][j] for i in range(10)]).argmax() 
                             for j in range(len(y_true))])

final_accuracy = accuracy_score(y_true, final_prediction)
print(f"Final accuracy after majority voting: {final_accuracy}")

In [12]:
print(total_results)
avg_results = total_results.mean(numeric_only=True)

print("\nAVERAGE RESULTS\n", avg_results)


In [13]:
final_metrics = {}
for dataset in [ds_tr, ds_vl]:
    y_true = np.array(all_y_true[dataset])
    y_pred = np.array(all_y_pred[dataset])
    
    acc = accuracy_score(y_true, y_pred) * 100
    uar = balanced_accuracy_score(y_true, y_pred) * 100
    
    final_metrics[f'{dataset}_ACC'] = acc
    final_metrics[f'{dataset}_UAR'] = uar

print("\nFinal Metrics:")
print(final_metrics)

Confusion matrix saved to: /media/carol/Data/Documents/Emo_rec/NewMel/IEMO_Mel_6/20250127_8/0/results/MSPI_Mel6_accuracy_41.60_UAR_42.01.png
Final accuracy after majority voting: 0.4202359579379328
   IEMO_Mel_6_ACC  IEMO_Mel_6_UAR  MSPI_Mel6_ACC  MSPI_Mel6_UAR
0       57.834101       57.032616      41.677353      41.441634
1       54.330709       56.816101      44.870480      40.437648
2       63.007160       60.673376      38.766350      42.550904
3       66.310160       56.423506      41.959477      41.289640
4       58.174905       50.375917      32.110798      39.425246
5       58.438819       55.290187      39.907669      41.631812
6       58.411215       62.652935      38.163632      44.243978
7       59.220779       57.054510      40.228264      42.044744
8       59.538784       62.884004      39.381893      43.156237
9       61.720430       62.722147      41.600410      42.005566

AVERAGE RESULTS
 IEMO_Mel_6_ACC    59.698706
IEMO_Mel_6_UAR    58.192530
MSPI_Mel6_ACC     39.866

In [14]:
model_info = {
    "Pretrain_file": pathstr,
    "Dataset Used": dataset_train,
    "Model Type": model_type,
    "Super Loss PARAMS": super_loss_params,
    "Speaker Disentanglement": Speaker_Disentanglement,
    "Entropy Curriculum Training": Entropy,
    "Column Trained on": column,
    "Test SpeakerID": speakers,
    "Angry Weight": angry_weight,
    "Happy Weight": happy_weight,
    "Neutral Weight": neutral_weight,
    "Sad Weight": sad_weight,
    "Weight decay": wt_dc, 
    "Avg Results": avg_results, 
    "total results": final_metrics
}
file_path = save_model_header(root_path, model_info)